# ファイルマネージャー

ファイルマネージャーでは、11_record_cameraや17_runで撮影したカメラ動画の編集をおこないます。不要な画像の削除や、新しいデータセットへの切出しができます。

In [1]:
import os

# WORKSPACEは、環境に合わせて書き直す!
WORKSPACE = None # <- 環境に合わせて修正してください。Noneを指定すると、Currentフォルダを参照

if WORKSPACE == None:
    current_path = os.getcwd()
else:
    current_path = WORKSPACE

In [3]:
import ipywidgets as widgets
#from jetcam.utils import bgr8_to_jpeg
import os
from IPython.display import display
from ipywidgets import Button, Layout
import re

WIDTH = 80
HEIGHT = 80

LOAD_DATASETS = []
LOAD_TASK_CUT = ['camera']

LOAD_TASK_DELETE = ['camera','dataset','detect','interactive']

current_path = os.getcwd()

SIZE = 10
no = 0

In [4]:
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [5]:
# 画像を表示するウィジェット
snapshot_widgets = []
for _ in range(SIZE):
    snapshot_widgets.append(widgets.Image(width=WIDTH, height=HEIGHT))

# 画像枚数
image_count_widget = widgets.IntText(description='画像枚数', value=0)

# 読込関連ウィジェット
load_button = widgets.Button(description='読込')
load_datasets_dropdown = widgets.Dropdown(options=LOAD_DATASETS, description='読込dataset')
load_task_dropdown = widgets.Dropdown(options=LOAD_TASK_CUT, description='読込task', index=0)

load_datasets_delete_dropdown = widgets.Dropdown(options=LOAD_DATASETS, description='読込dataset')
load_task_delete_dropdown = widgets.Dropdown(options=LOAD_TASK_DELETE, description='読込task', index=0)

# 画像ナビゲーションボタン
prev_image_button = widgets.Button(description='<')
next_image_button = widgets.Button(description='>')

# 更新ボタン
update_button = widgets.Button(description='更新')

# 範囲指定ウィジェット（共通）
start_index_widget = widgets.IntText(description='開始位置', value=0)
end_index_widget = widgets.IntText(description='終了位置', value=SIZE)

# 削除関連ウィジェット
delete_start_index_widget = widgets.IntText(description='削除開始位置', value=0)
delete_end_index_widget = widgets.IntText(description='削除終了位置', value=SIZE)
delete_button = widgets.Button(description='削除')
all_delete_button = widgets.Button(description='全削除')

# 切出し関連ウィジェット
cut_datasets_input = widgets.Text(description='切出dataset')
cut_task_dropdown = widgets.Dropdown(options=LOAD_TASK_CUT, description='切出task', index=0)
cut_start_index_widget = widgets.IntText(description='切出開始位置', value=0)
cut_end_index_widget = widgets.IntText(description='切出終了位置', value=SIZE)
cut_button = widgets.Button(description='指定区間を切出し')

In [6]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = widgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [9]:
import os
import cv2
import numpy as np
import shutil

def extract_numbers(filename):
    matches = re.findall(r'(\d+)', filename)
    if matches and len(matches) >= 3: 
        return int(matches[-1])  
    else:
        return float('inf')

def get_file_names(path):
    
    image_names = []
    
    try:
        file_names = os.listdir(path)
        file_names = [os.path.join(path, file_name) for file_name in file_names]
        image_names = sorted(file_names, key=lambda f: extract_numbers(os.path.basename(f)))
        image_names = [f for f in image_names if os.path.splitext(f)[1].lower() == ".jpg"]
    except Exception as e:
        write_log(f"Error: {e}")
    
    return image_names

def get_dirs(path):
    # ディレクトリ内のすべてのエントリを取得
    files = os.listdir(path)
    
    # ディレクトリのみをフィルタリング
    dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
    
    # 除外するディレクトリ名のリスト
    exclude_dirs = {".ipynb_checkpoints", ".DS_Store"}
    
    # 除外処理を実行
    dirs = [
        d for d in dirs
        if d not in exclude_dirs and not d.endswith('.zip')
    ]
    
    # ソートして返す
    dirs = sorted(dirs)
    return dirs

def next_pic(c):
    global no, start_index_widget, end_index_widget, last_no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_flag = True
    no += SIZE
    if no < last_no:
        start_number = no
        end_number = no + SIZE
        start_index_widget.value = start_number
        end_index_widget.value = end_number
        load_img(no)
    else:
        no -= SIZE
        next_image_button.disabled = False
        prev_image_button.disabled = False
    
def before_pic(c):
    global no, start_index_widget, end_index_widget, last_no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_flag = True
    no -= SIZE
    if no < 0:
        no = 0
    start_number = no
    end_number = no + SIZE
    start_index_widget.value = start_number
    end_index_widget.value = end_number
    load_img(no)
    
def update_pic(c):
    global no, start_index_widget, end_index_widget, last_no
    no = start_index_widget.value
    if no < 0:
        no = 0
    elif no < last_no:
        start_number = no
        end_number = no + SIZE
        start_index_widget.value = start_number
        end_index_widget.value = end_number
        load_img(no)

def load_img(no):
    global snapshot_widgets, last_no, current_path
    
    xy_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "xy")     
    xy_filenames = get_file_names(xy_path)
    
    image_count_widget.value = len(xy_filenames)
    last_no = len(xy_filenames)
    for i in range(SIZE):
        if no + i < last_no:
            try:
                xy_name = xy_filenames[no + i]
                img = cv2.imread(xy_name)
                snapshot_widgets[i].value = bgr8_to_jpeg(img)
            except:
                black_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
                snapshot_widgets[i].value = bgr8_to_jpeg(black_image)
        else:
            black_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
            snapshot_widgets[i].value = bgr8_to_jpeg(black_image)
        
    next_image_button.disabled = False
    prev_image_button.disabled = False
    
def change_load_task(c):
    global current_path
    try:
        path = os.path.join(current_path, load_task_dropdown.value)
        dirs = get_dirs(path)
        load_datasets_dropdown.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_dropdown.options = []
        
def change_load_task_delete(c):
    global current_path
    try:
        path = os.path.join(current_path, load_task_delete_dropdown.value)
        dirs = get_dirs(path)
        load_datasets_delete_dropdown.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_delete_dropdown.options = []

def load_pic(c):
    global no
    no = 0
    start_index_widget.value = 0
    end_index_widget.value = SIZE
    load_img(no)
    
def delete_pic(c):
    global current_path
    start_pos = delete_start_index_widget.value
    end_pos = delete_end_index_widget.value
    write_log(f"{start_pos}から{end_pos}まで削除します")
    
    xy_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "xy")
    xy_filenames = get_file_names(xy_path)
    speed_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "speed")
    speed_filenames = get_file_names(speed_path)
    
    start_pos = max(start_pos, 0)
    end_pos = min(end_pos, len(xy_filenames))
    write_log(f"{start_pos}から{end_pos}2")
    for h in range(start_pos, end_pos):
        try:
            os.remove(speed_filenames[h])
            if h % 100 == 0:
                write_log(f"speedの{h+1}枚目 削除しました。")
        except Exception as e:
            #write_log(f"Error: {e}")
            pass
        
    for i in range(start_pos, end_pos):
        try:
            os.remove(xy_filenames[i])
            if i % 100 == 0:
                write_log(f"xyの{i+1}枚目/{len(xy_filenames)}枚中 削除中")
        except IndexError:
            write_log(f"No image to delete at position {i}")
        except OSError as e:
            write_log(f"Error deleting {xy_filenames[i]}: {e}") 
    write_log(f"{start_pos}から{min(end_pos, len(xy_filenames))}までを削除しました。")
   
    xy_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "xy")     
    xy_filenames = os.listdir(xy_path)
    xy_filenames = [os.path.join(xy_path, file_name) for file_name in xy_filenames if file_name.endswith('.jpg')]
    xy_filenames.sort(key=lambda x: os.path.getmtime(x))
    image_count_widget.value = len(xy_filenames)
    last_no = len(xy_filenames)
        
def cut_dataset(c):
    global current_path

    start_pos = cut_start_index_widget.value
    end_pos = cut_end_index_widget.value
    task = load_task_dropdown.value
    new_dataset_name = cut_datasets_input.value

    source_xy_path = os.path.join(current_path, task, load_datasets_dropdown.value, "xy")
    source_speed_path = os.path.join(current_path, task, load_datasets_dropdown.value, "speed")

    dest_xy_path = os.path.join(current_path, task, new_dataset_name, "xy")
    if not os.path.exists(dest_xy_path):
        os.makedirs(dest_xy_path)
    dest_speed_path = os.path.join(current_path, task, new_dataset_name, "speed")
    if not os.path.exists(dest_speed_path):
        os.makedirs(dest_speed_path)

    xy_filenames = get_file_names(source_xy_path)
    speed_filenames = get_file_names(source_speed_path)

    for h in range(start_pos, min(end_pos, len(speed_filenames))):
        try:
            dest_file_path = os.path.join(dest_speed_path, os.path.basename(speed_filenames[h]))
            shutil.copyfile(speed_filenames[h], dest_file_path)
            os.remove(speed_filenames[h])
            if h % 100 == 0:
                write_log(f"speedの{h+1}枚目/{len(speed_filenames)}枚中 切出中")
        except:
            pass
        
    for i in range(start_pos, min(end_pos, len(xy_filenames))):
        try:
            dest_file_path = os.path.join(dest_xy_path, os.path.basename(xy_filenames[i]))
            shutil.copyfile(xy_filenames[i], dest_file_path)
            os.remove(xy_filenames[i])
            if i % 100 == 0:
                write_log(f"xyの{i+1}枚目/{len(xy_filenames)}枚中 切出中")
        except IndexError:
            write_log(f"No image to move at position {i}")
        except OSError as e:
            write_log(f"Error moving {xy_filenames[i]}: {e}")
    write_log(f"{start_pos}から{min(end_pos, len(xy_filenames))}までを切出しました。")
        
    
        
    load_img(0)
    update_image_count()
    
def update_image_count():
    global image_count_widget, last_no, current_path
    xy_path = os.path.join(current_path, load_task_dropdown.value, load_datasets_dropdown.value, "xy")
    if os.path.exists(xy_path):
        xy_filenames = get_file_names(xy_path)
        image_count_widget.value = len(xy_filenames)
        last_no = len(xy_filenames)
    else:
        image_count_widget.value = 0
        last_no = 0

def delete_all(c):
    global current_path
    
    path = os.path.join(current_path, load_task_delete_dropdown.value, load_datasets_delete_dropdown.value)

    if os.path.isdir(path):
        shutil.rmtree(path)
        write_log(f"{path}を全削除しました。")
    else:
        write_log("指定されたパスはディレクトリではありません。")
    path_dirs = os.path.join(current_path, load_task_delete_dropdown.value)
    dirs = get_dirs(path_dirs)
    load_datasets_delete_dropdown.options = dirs
        
    
    
change_load_task(LOAD_TASK_CUT[0])        
load_task_dropdown.observe(change_load_task, names='value')
change_load_task_delete(LOAD_TASK_DELETE[0])        
load_task_delete_dropdown.observe(change_load_task_delete, names='value')

load_button.on_click(load_pic)
prev_image_button.on_click(before_pic)
next_image_button.on_click(next_pic)
update_button.on_click(update_pic)
delete_button.on_click(delete_pic)
cut_button.on_click(cut_dataset)
all_delete_button.on_click(delete_all)

In [10]:
separator = widgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = widgets.HTML('<b>■ 不要データの削除</b><br>【削除開始位置】,【削除終了位置】を指定して【削除ボタン】で不要データを削除します。')
title2 = widgets.HTML('<b>■ データセットの切出</b><br>【切出開始位置】,【切出終了位置】を指定して、【指定区間を切出し】ボタンで新しいデータセットに切出します。')
title３ = widgets.HTML('<b>■ データセットの削除</b><br>選択したデータセットを削除します。')

data_collection_widget = widgets.VBox([
    separator,
    title1,
    widgets.HBox(snapshot_widgets),
    widgets.HBox([load_datasets_dropdown, load_task_dropdown, load_button]),
    widgets.HBox([prev_image_button, next_image_button, image_count_widget]),
    widgets.HBox([start_index_widget, update_button, end_index_widget]),
    widgets.HBox([delete_start_index_widget, delete_end_index_widget, delete_button]),
    process_widget,
    separator,
    title2,
    widgets.HBox([cut_datasets_input, cut_task_dropdown, cut_button]),
    widgets.HBox([cut_start_index_widget, cut_end_index_widget]),
    process_widget,
    separator,
    title3,
    widgets.HBox([load_datasets_delete_dropdown, load_task_delete_dropdown, all_delete_button]),
    process_widget,
])

display(data_collection_widget)
try:
    load_img(0)
except:
    write_log(f"データが不正です。")